## Installing dependencies

In [20]:
!pip install weaviate-client
!pip install -U sentence-transformers
!pip install -q pyvi


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\jiji\Documents\Code\RAG-pipeline\venv\Lib\site-packages\IPython\utils\_process_win32.py:138: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  res = process_handler(cmd, _system_body)
c:\Users\jiji\Documents\Code\RAG-pipeline\venv\Lib\site-packages\IPython\utils\_process_win32.py:138: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  res = process_handler(cmd, _system_body)
c:\Users\jiji\Documents\Code\RAG-pipeline\venv\Lib\site-packages\IPython\utils\_process_win32.py:138: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  res = process_handler(cmd, _system_body)


     ---------------------------------------- 0.0/40.9 kB ? eta -:--:--
     -------------------------------------- 40.9/40.9 kB 958.4 kB/s eta 0:00:00
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 3.4 MB/s eta 0:00:00
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ---------------------------------------- 60.9/60.9 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB 2.0 MB/s eta 0:00:00
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   --


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Connecting to Weaviate

In [28]:
import weaviate

client = weaviate.connect_to_local("localhost")

In [30]:
from weaviate.classes.config import Configure, Property, DataType
try:
    client.collections.create(
        name="Document",
        vectorizer_config=Configure.Vectorizer.none(),
        properties=[Property(name="text", data_type=DataType.TEXT)],
    )
except Exception as e:
    print(f"Error creating collection: {e}")

Error creating collection: Collection may not have been created properly.! Unexpected status code: 422, with response body: {'error': [{'message': 'class name Document already exists'}]}.


### Closing the connection (if needed)

In [22]:
client.close()

## Embedding

### Quick test

In [23]:
import time
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize

start = time.time()
sentences = ["Hà Nội là thủ đô của Việt Nam", "Đà Nẵng là thành phố du lịch"]
tokenizer_sent = [tokenize(sent) for sent in sentences]
print(f"Tokenized sentences in {time.time() - start:.2f} seconds: {tokenizer_sent}")

start = time.time()
model = SentenceTransformer('dangvantuan/vietnamese-embedding')
embeddings = model.encode(tokenizer_sent)
print(f"Embedding in {tokenizer_sent} in {time.time() - start:.2f} seconds: {embeddings}")



Tokenized sentences in 0.00 seconds: ['Hà_Nội là thủ_đô của Việt_Nam', 'Đà_Nẵng là thành_phố du_lịch']
Embedding in ['Hà_Nội là thủ_đô của Việt_Nam', 'Đà_Nẵng là thành_phố du_lịch'] in 4.12 seconds: [[ 0.29940763  0.16308057 -0.24620049 ... -0.00254593  0.03522857
   0.38378468]
 [ 0.20620261  0.1663139  -0.1470831  ...  0.31187794 -0.61788845
  -0.37897834]]


### Real embedding function

In [24]:
def embed(text):
    return model.encode(text).tolist()

### Insert data to Weaviate

In [64]:
client.collections.delete("Document")

In [ ]:
def text_to_vector(texts):
    for text in texts:
        vector = embed(text)
        start = time.time()
        client.collections.get("Document").data.insert(
            properties={"text": text}, vector=vector
        )
    print(f"Inserted {len(texts)} texts in {time.time() - start:.2f} seconds: {text[:30]}...")

In [66]:
# Example texts in Vietnamese about cities but with made up facts to check if the model
#  really extracts the meaning of the sentences.
texts = [
    "Thành phố Sương Mù, hay còn gọi là Fogsville, là một đô thị cổ kính nằm sâu trong dãy núi Lạc Lối. Nổi tiếng với những con đường lát đá phủ đầy rêu phong và những tòa nhà chọc trời bằng gỗ tuyết tùng, Sương Mù là quê hương của loài chim 'Thiên Hạc Lửa' quý hiếm, một loài chim có khả năng biến mọi thứ xung quanh thành vàng chỉ bằng một tiếng hót. Cư dân ở đây tin rằng việc chạm vào lông của Thiên Hạc Lửa sẽ mang lại may mắn, nhưng phải cẩn thận vì nó cũng có thể biến bạn thành một cây nấm khổng lồ.",
    "Thủ đô Biển Cả của vương quốc Thủy Tề là một thành phố dưới nước hoàn toàn, được xây dựng từ những viên ngọc trai khổng lồ và san hô phát sáng. Không khí ở đây được tạo ra bởi những 'bong bóng âm nhạc' đặc biệt, mỗi bong bóng mang một giai điệu riêng và cung cấp oxy cho cư dân. Phương tiện giao thông chính là 'cá voi điện từ', có khả năng dịch chuyển tức thời qua các đường hầm nước ngầm. Biển Cả là nơi duy nhất trên thế giới có 'cửa hàng thời trang rong biển', nơi các nhà thiết kế tạo ra những bộ trang phục lộng lẫy từ các loài rong biển quý hiếm.",
    "Thành phố Bay Lượn, nằm trên một hòn đảo trôi nổi giữa không trung, được mệnh danh là 'Thiên Đường Của Mây'. Cư dân ở đây di chuyển bằng những chiếc dù lượn cá nhân được gắn động cơ năng lượng gió và những 'đám mây taxi' đặc biệt có thể đưa bạn đến bất cứ đâu. Mỗi buổi sáng, mặt trời mọc ở Bay Lượn sẽ phát ra những tia sáng có màu sắc khác nhau tùy thuộc vào tâm trạng của người dân. Người ta đồn rằng, ai có thể đếm được hết số cây cầu mây ở thành phố này sẽ tìm thấy kho báu bị chôn giấu từ hàng ngàn năm trước.",
    "Đảo Khủng Long, một thành phố nằm ẩn mình trong một miệng núi lửa đã tắt, là nơi sinh sống hòa bình của con người và những chú khủng long thông minh. Khủng long ở đây không chỉ là vật nuôi mà còn là những kiến trúc sư tài ba, giúp con người xây dựng những công trình vĩ đại từ xương rồng hóa thạch. Thư viện trung tâm của thành phố được quản lý bởi một con khủng long T-Rex khổng lồ có khả năng ghi nhớ mọi cuốn sách từng được viết. Điều đặc biệt là, nếu bạn nói dối trên Đảo Khủng Long, móng chân của bạn sẽ tự động dài ra một inch.",
    "Thành phố Ánh Sáng của vùng đất Sắc Màu là một đô thị rực rỡ với những con đường được lát bằng tinh thể phát quang và những tòa nhà thay đổi màu sắc theo cảm xúc của người bên trong. Ở đây, thay vì tiền tệ, người dân sử dụng 'những nụ cười' để mua bán, mỗi nụ cười có một giá trị khác nhau tùy thuộc vào mức độ chân thành. Vào mỗi đêm trăng tròn, cả thành phố sẽ tổ chức 'Lễ Hội Đèn Lồng Sao Băng', nơi hàng ngàn đèn lồng được thả lên trời, mang theo những ước mơ của người dân và biến chúng thành hiện thực nếu bạn tin tưởng đủ mạnh.",
]

text_to_vector(texts)

Inserted 5 texts in 0.05 seconds: Thành phố Ánh Sáng của vùng đấ...


### Query

In [67]:
from weaviate.classes.query import MetadataQuery

query = "Tinh thể?"
query_tokenized = tokenize(query)
query_vector = model.encode(query_tokenized).tolist()

result = client.collections.get("Document").query.near_vector(
    near_vector=query_vector,
    limit=5,
    return_metadata=MetadataQuery(distance=True)
)

retrieved_objects = [obj for obj in result.objects]

print("Query results:")
for i, obj in enumerate(retrieved_objects, 1):
    print(f"{i}. Dist: {obj.metadata.distance} - {obj.properties['text'][:140]}...")
    

Query results:
1. Dist: 0.8997213840484619 - Thành phố Ánh Sáng của vùng đất Sắc Màu là một đô thị rực rỡ với những con đường được lát bằng tinh thể phát quang và những tòa nhà thay đổi...
2. Dist: 0.9284766316413879 - Thủ đô Biển Cả của vương quốc Thủy Tề là một thành phố dưới nước hoàn toàn, được xây dựng từ những viên ngọc trai khổng lồ và san hô phát sá...
3. Dist: 0.9458974003791809 - Thành phố Bay Lượn, nằm trên một hòn đảo trôi nổi giữa không trung, được mệnh danh là 'Thiên Đường Của Mây'. Cư dân ở đây di chuyển bằng nhữ...
4. Dist: 0.963503897190094 - Thành phố Sương Mù, hay còn gọi là Fogsville, là một đô thị cổ kính nằm sâu trong dãy núi Lạc Lối. Nổi tiếng với những con đường lát đá phủ ...
5. Dist: 1.0627187490463257 - Đảo Khủng Long, một thành phố nằm ẩn mình trong một miệng núi lửa đã tắt, là nơi sinh sống hòa bình của con người và những chú khủng long th...
